# Text Classifiction

In [1]:
import pandas as pd
data=pd.read_csv('final_dataset_basicmlmodel.csv')
data.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


label is target variable,
tweet contains text

There is noise in the text such as hashtags and unicode characters that need to be cleaned

# Text cleaning

In [8]:
import re
def clean_text(text):
 text = re.sub(r'[^a-zA-Z\']', ' ', text)
 text = re.sub(r'[^\x00-\x7F]+', '', text)
 text = text.lower()
 return text

In [10]:
data['clean_text']=data.tweet.apply(lambda x: clean_text(x))

In [32]:
from wordcloud import STOPWORDS
def gen_text(text):
 word_list=[]
 for word in text.split():
    word_list.extend(word)
    
    word_freq=pd.Series(word_list).value_counts()
    word_freq=word_freq.drop(STOPWORDS,errors='ignore')
    
    return word_freq

In [41]:
def nef(words):
    for nword in words:
        if nword in ['n','no','not','non']:
         return 1
    else:
        return 0
   
def rare(words,rare_100):
        for rword in words:
            if rword in rare_100:
                return 1
            else:
                return 0

def question(words):
    for qword in words:
        if qword in ['when','what','which','where','how']:
         return 1
    else:
        return 0

In [42]:
word_freq=gen_text(data.clean_text.str)
rare_100=word_freq[-100:]
data['word_count']=data.clean_text.str.split().apply(lambda x: len(x))
data['char_count']=data.clean_text.apply(lambda x: len(x))
data['negative']=data.clean_text.str.split().apply(lambda x: nef(x))
data['rare']=data.clean_text.str.split().apply(lambda x: rare(x,rare_100))
data['question']=data.clean_text.str.split().apply(lambda x: question(x))

In [43]:
data.head()

,id,label,tweet,clean_text,word_count,char_count,negative,rare,question
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is s...,18,102,0,1,1
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can't us...,19,122,0,1,0
2,3,0,bihday your majesty,bihday your majesty,3,21,0,0,0
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ...,12,86,0,0,0
4,5,0,factsguide: society now #motivation,factsguide society now motivation,4,39,0,0,0


# Split

In [47]:
from sklearn.model_selection import train_test_split
x=data[['word_count','char_count','negative','rare','question']]
y=data[['label']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=27)

# Train

In [48]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model = model.fit(x_train, y_train)
pred = model.predict(x_test)

G:\aNAA\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Evaluate

In [49]:
from sklearn.metrics import accuracy_score

print("Accuracy:", accuracy_score(y_test, pred)*100, "%")

Accuracy: 66.0952380952381 %
